<a href="https://colab.research.google.com/github/MinhDinh-4869/RNN-Test/blob/main/rnn_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
 
import numpy as np

In [ ]:
#create an dictionary for one hot vector
text = ['hey how are you ?','good i am fine','have a nice day', 'hello mother fucker', 'how are you today ?', 'i am good', 'i love you', 'my name is', 'what your name ?', 'what are you doing ?']

#join all the senteces togenther and extract the unique characters from the 
#combined sentences chars

chars = set(''.join(text))

#creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

#creating a dictionary that maps characters to the integers
char2int = {char: ind for ind, char in int2char.items()}
print(char2int)

{'k': 0, 'a': 1, 'r': 2, 'i': 3, 'o': 4, ' ': 5, 't': 6, 'c': 7, 'h': 8, '?': 9, 'd': 10, 'e': 11, 'y': 12, 'g': 13, 'm': 14, 'f': 15, 'v': 16, 's': 17, 'n': 18, 'w': 19, 'l': 20, 'u': 21}


In [ ]:
#finding the length of the longest string in our data
maxlen = len(max(text, key = len))

#padding

#A simple loop that loops through the list of sentences and adds a ' ' white
#space until the length of the sentences matches the length of the longest sentece

for i in range(len(text)):
  while(len(text[i]) < maxlen):
    text[i]+= ' '


In [ ]:
#Now we are going to predict the next character in the sequece at each time step, we will have to dived each sentence into
#1. Input data: the last input charactyer should be excluded as it does not need to be fed into the model
#2. target/ground truth label
#e.g: in: hey how are yo --> ey how are you
# input a word and predict the next word, and the size of the input sentence is fixed
#as the network size is fixed. then the first character of the ground truth is missing

input_seq = []
target_seq = []

for i in range(len(text)):
  #remove the last character
  input_seq.append(text[i][:-1]) #[-1] is the last item in a list

  #remove the first character
  target_seq.append(text[i][1:])

print()


In [ ]:
#convert to integer vectors
print(input_seq)

for i in range(len(text)):
  input_seq[i] = [char2int[character] for character in input_seq[i]]
  target_seq[i] = [char2int[character0] for character0 in target_seq[i]]



['hey how are you ?  ', 'good i am fine     ', 'have a nice day    ', 'hello mother fucker', 'how are you today ?', 'i am good          ', 'i love you         ', 'my name is         ', 'what your name ?   ', 'what are you doing ']


In [ ]:
dict_size = len(char2int)
seq_len = maxlen - 1 # length of a processed sequence = len - 1
batch_size = len(text) # the number of sentences that we defined and are going to be fed into the model as a batch

def one_hot_encode(sequence, dict_size, seq_len, batch_size): #for 1 sequence: target or input
  #creating a multi dimensial array of zeros with the desired output shape
  features  = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)

  #replacing the 0 that the relevant character
  for i in range(batch_size):
    for u in range(seq_len):             #   i   u
      features[i, u, sequence[i][u]] = 1 #e.g[[4,16,3,9,4,8,2,9,15,6,16,9,3,8]] = [[h,e,y,h,o,w,a,r,e,y,o]]
  return features

print(one_hot_encode(input_seq, dict_size, seq_len, batch_size))
  

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [ ]:
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
#target_seq = one_hot_encode(target_seq, dict_size, seq_len, batch_size) !! dont need this

input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [ ]:
print(target_seq.shape)

torch.Size([3, 14])


In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)

        out = self.fc(out)

        #out = out.reshape(x.shape)

        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [ ]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
n_epochs = 300
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    input_seq.to(device)
    output, hidden = model(input_seq)
    #loss = criterion(output, target_seq)#).view(-1).long())
    loss = criterion(output, target_seq.view(-1).long())
    #output shape: (batch_size x input_size, dict_size) --> (42, 17)
    #target shape: (batch_size, input_size) -> (3, 14)
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/300............. Loss: 2.3489
Epoch: 20/300............. Loss: 2.2215
Epoch: 30/300............. Loss: 2.0669
Epoch: 40/300............. Loss: 1.8596
Epoch: 50/300............. Loss: 1.6099
Epoch: 60/300............. Loss: 1.3811
Epoch: 70/300............. Loss: 1.1876
Epoch: 80/300............. Loss: 1.0202
Epoch: 90/300............. Loss: 0.8638
Epoch: 100/300............. Loss: 0.7351
Epoch: 110/300............. Loss: 0.6275
Epoch: 120/300............. Loss: 0.5378
Epoch: 130/300............. Loss: 0.4621
Epoch: 140/300............. Loss: 0.3978
Epoch: 150/300............. Loss: 0.3471
Epoch: 160/300............. Loss: 0.3078
Epoch: 170/300............. Loss: 0.2727
Epoch: 180/300............. Loss: 0.2455
Epoch: 190/300............. Loss: 0.2224
Epoch: 200/300............. Loss: 0.2028
Epoch: 210/300............. Loss: 0.1977
Epoch: 220/300............. Loss: 0.1856
Epoch: 230/300............. Loss: 0.1664
Epoch: 240/300............. Loss: 0.1546
Epoch: 250/300...........

In [ ]:
from torch.functional import Tensor
# This function takes in the model and character as arguments and returns the next character prediction and hidden state
def test_predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    return prob

# This function takes in the model and character as arguments and returns the next character prediction and hidden state
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [ ]:
prob = test_predict(model, 'good did')
for i in range(prob.shape[0]):
  char_ind = np.argmax(prob.numpy()[i])
  print(int2char[char_ind], np.max(prob.numpy()[i]))


#print(int2char[char_ind])


k 4.759894e-05
k 0.23260434
k 0.00020718614
k 9.54858e-05
k 0.0008086724
k 0.0023159874
k 5.7562293e-06
k 0.00015983757
k 1.165815e-05
k 0.011992644
k 2.4871073e-05
k 0.65938413
k 0.066552036
k 0.013119103
k 6.2544e-05
k 0.00096558844
k 0.000117105614
k 0.0007169355
k 0.0048768814
k 0.00097007054
k 0.004290256
k 0.00067125907


In [ ]:
# This function takes the desired output length and input characters as arguments, returning the produced sentence
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [ ]:
sample(model, 20, 'what do')

'what doy am good ing'